# WORKSHOP 2

------------

GRUPO 4:

Digital House

link git hub: https://github.com/grupo4dhds/grupo4_git.git

--------------

## CONTENIDO

**items celda 4:**
	
	- 1. COLUMNAS REDUNDANTES
		- 1.1 Recupero place_name
		- 1.2 Filtrado columnas de trabajo
		- GUARDADO: data_cruda

	- 2. ANALISIS DE COTIZACION
		- 2.1 Creacion de columna soporte previsoria
		- 2.2 Filtrado de datos ilogicos
		- 2.3 Creacion de variable soporte - tipo de cambio
		- 2.4 Unificacion del tipo de cambio en el precio
		- 2.5 Filtrado y Recupero de price_usd_per_m2

	- 3. REACIONDICIONAMIENTO DEL DATASET
		- 3.1 Filtrado de columnas
		- 3.2 Obtencion de muestra: currency_nan
		- 3.3 Obtencio de muestra: surface_nan
		- GUARDADO: limpieza_1

**items celda 5:**

	- 4. LIMPIEZA POR COUNT Y CV
		- 4.1 Filtrado por cantidad de publicaciones máximas en state_name
		- 4.2 Filtrado por cantidad de publicaciones mínimas en place_name
		- GUARDADO: limpieza_2
		- 4.3 Filtrado por coeficiente de variación (1)
		- GUARDADO: limpieza_3
		- 4.4 Filtrado por coeficiente de variación (2)
		- GUARDADO: limpieza_4

**items celda 6:**

	- 5. FILTRADO DE OUTLIERS POR SUPERFICIE TOTAL 
		- GUARDADO: limpieza_5_aux
		- GUARDADO: limpieza_5

	- 6. RECUPERO DE LA DATA SUPERFICIE TOTAL
		- GUARDADO: recupero_5
	
**items celda 7:**

	- 7. FILTRADO DE OUTLIERS POR PRECIO USD POR M2 
		- GUARDADO: limpieza_6_aux
		- GUARDADO: limpieza_6
        
--------
grupo4dhds@gmail.com / a123b456c789

grupo4dhds / a123b456c789!_!

-----------

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
data = pd.read_csv('properatti.csv')
Dicto = {}

warnings.filterwarnings('ignore')

In [3]:
def data_info(data, name='data'):
    df = pd.DataFrame(pd.Series(data.columns))
    df.columns = ['columna']
    df.columns.name = f'df de {name}'
    df.index.name = 'index'
    df['Nan'] = data.isna().sum().values
    df['dtype']  = data.dtypes.values
    df['count'] = data.count().values
    df['pct_nan'] = round(df['Nan']/data.shape[0]*100,2)
    df['count_unique'] = [len(data[elemento].value_counts()) for elemento in data.columns]
    return df

def explore_count(frame, label_1):
    df = pd.DataFrame(index = pd.Series(frame[label_1].unique()))
    frame['all_one'] = 1
    #cantidad de datos publicados
    df = df.join(frame.groupby(label_1)['all_one'].count())
    df.columns = ['total']
    frame.drop('all_one', axis=1, inplace=True)
    return df

def explore_count_by(frame,label_1, label_2, df):
    # cantidad de tipo apartment
    set_check = list(set(frame[label_2]))
    frame['all_one'] = 1
    for i,propiedad in enumerate(set_check):
        serie_check =frame[frame[label_2]==propiedad].groupby(label_1)['all_one'].count()
        df['count_'+propiedad] = 0 + serie_check
    frame.drop('all_one', axis=1, inplace=True)
    return df

def explore_statics_count(frame, label_1, label_2, label_3, df):
    # precio promedio y la mediana de precio para state_name, property_type
    data_check = frame.groupby([label_1, label_2])[label_3].aggregate(['mean', 'std', 'median'])
    data_check.reset_index(inplace=True)
    data_check = data_check.pivot_table(index=label_1, columns=label_2)
    set_check = list(set(data[label_2]))
    for propiedad in set_check:
        df[propiedad+'_mean_'+label_3] = 0 + data_check.loc[:]['mean'][propiedad]
        df[propiedad+'_std_'+label_3] = 0 + data_check.loc[:]['std'][propiedad]
        df[propiedad+'_median_'+label_3] = 0 + data_check.loc[:]['median'][propiedad]
    # dispersion de los precios
    for propiedad in set_check:
        df['CV_'+propiedad+'_'+label_3] = df[propiedad+'_std_'+label_3]/df[propiedad+'_mean_'+label_3]
    return df

def rango_extremos(frame, label, quant_list):
        for i,v in enumerate(quant_list):
            quant_list[i] = v /100
        
        rango_outlier = frame[label].quantile(quant_list)
        rango_outlier = list(rango_outlier)
        print ('range_outlier: type: {}, len: {}, value: {}'.format(type(rango_outlier),len(rango_outlier),rango_outlier))
        return rango_outlier
    
def mascara_outlier_index_list(frame, label, rango_outlier): # el problema que tengo es sacarme solo True de encima y quedarme con el indice de los False - 
    mascara_outlier = (frame[label].dropna() >= rango_outlier[0]) & (frame[label].dropna() <= rango_outlier[1])  #ME QUEDO CON EL RANGO FALSE
    #resulta: mascara booleana quiero quedarme con el indice de los False
    lista_elementos = mascara_outlier.values
    lista_indices = mascara_outlier.index
    lista_elementos = list(lista_elementos)
    lista_indices = list(lista_indices)
    lista_result = []
    for i, booleano in enumerate(lista_elementos):
        if booleano == False:
            lista_result.append(lista_indices[i])
    return lista_result

def check_range(frame, label, rango):
    mask_data = (frame[label] <= rango[0]) | (frame[label] >= rango[1])  # tomo True los valores que caen fuera del rango
    # necesito solo los indices que sean True
    lista_elementos = list(mask_data.values)
    lista_indices = list(mask_data.index)
    lista_result = []
    for i, booleano in enumerate(lista_elementos):
        if booleano == True:
            lista_result.append(lista_indices[i])
    return lista_result

def cambia_nombre_columnas(data=data, **kwarg):
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
    for key in kwarg.keys():
        data.rename(kwarg[key], axis=1, inplace=True)
    return data

def cambia_orden_columnas(check_list, data=data, safe=1): # safe: es para conservar la primera columna con indice[0]
    label_list = list(data.columns)
    for i in range(len(check_list)):
        label_list.remove(check_list[i])
        label_list.insert(i+safe, check_list[i])
        data = data.reindex(label_list, axis=1)
    return data

In [4]:
# 1. COLUMNAS REDUNDANTES
# 1.1 Recupero datos colgados
data.loc[data.place_name.isna(),'place_name'] = 'Tigre'
# 1.2 COLUMNAS DE TRABAJO
columnas = ['property_type', 'place_name', 'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2']
data = data.filter(columnas, axis=1)
# # GUARADADO DE LA DATA # #
df = data_info(data, 'data_cruda')
Dicto['data_cruda'] = df.to_dict(), data.to_dict()

# 2. ANALISIS DE COTIZACION
# 2.1: creacion de una columna soporte de superficie
mask_data = data.surface_total_in_m2.notna() & data.surface_covered_in_m2.notna()
mask_data = data[mask_data].surface_total_in_m2 >= data[mask_data].surface_covered_in_m2 # comprobamos que haya logica Total > Cubierto
data['surface_total_in_m2'].fillna(0, inplace=True)
data['surface_covered_in_m2'].fillna(0, inplace=True)
data['surface_total_in_m2_mod'] = np.maximum(data.surface_total_in_m2, data.surface_covered_in_m2)
data['surface_total_in_m2_mod'] = data['surface_total_in_m2_mod'].replace(0,np.nan)
data['surface_total_in_m2'] = data['surface_total_in_m2'].replace(0,np.nan)
data['surface_covered_in_m2'] = data['surface_covered_in_m2'].replace(0,np.nan)

# 2.2 #comprobamos que hay valores ilogicos porque price_usd_per_m2 es mayor a price_per_m2 que esta en pesos
mask_data = data.currency == 'ARS'
data.drop((data[mask_data].loc[data.price_per_m2 < data.price_usd_per_m2]).index, inplace=True)

# 2.3: DEFINICION DEL TIPO DE CAMBIO
tc = np.mean(data.price_aprox_local_currency/data.price_aprox_usd) # tc promedio 17.6444

# 2.4: PASAR LA COLUMNA price A UN SOLO TIPO DE CAMBIO
mask_data = data.currency == 'ARS'
data.loc[mask_data, 'price'] = data[mask_data].price / tc
data.loc[mask_data, 'currency'] = 'USD'
mask_data = data.loc[:,'currency'] != 'USD'
mask_data = data.loc[mask_data,'currency'].dropna().index
data.drop(mask_data, inplace=True)

# 2.5: ELIMINAMOS 870 VALORES INCOHERENTES Y RECUPERO DE PRICE_USD_PER_M2
data['price_usd_per_m2_recovery'] = data['price'] / data['surface_total_in_m2_mod']
data['diff_pct_change'] = (data['price_usd_per_m2_recovery'] / data['price_usd_per_m2'] -1 )*100
# ' Elimino del dataset de trabajo los que tenian desvios de +- 4.75%'
mask_data = data.loc[(data['diff_pct_change'] < -4.75) | (data['diff_pct_change'] > 4.75)].index
data.drop(mask_data, inplace=True)
data['price_usd_per_m2'].fillna(data['price_usd_per_m2_recovery'], inplace=True)

# 3 REACONDICIONAMIENTO DEL DATASET
# 3.1: FILTRADO DE COLUMNAS
columnas = ['diff_pct_change','price_usd_per_m2_recovery', 'surface_total_in_m2_mod', 'price_aprox_usd', 'price_per_m2', 'price_aprox_local_currency']
data.drop(columnas, axis=1, inplace=True)
    
# 3.2: TOMAMOS MUESTRA DE LOS CURRENCY NANS
df = data_info(data[data.currency.isna()], 'currency_nan')
Dicto['currency_nan'] = df.to_dict(), data[data.currency.isna()].to_dict()

# 3.3: TOMAMOS MUESTRA DE SURFACE_COVERED ISNA() Y SURFACE_TOTAL ISNA()
mask_1 = data.surface_total_in_m2.isna()
mask_2 = data.surface_covered_in_m2.isna()
df = data_info(data[mask_1 & mask_2], 'surface_nan')
Dicto['surface_nan'] = df.to_dict(), data[mask_1 & mask_2].to_dict()

# # GUARDADO DE LA DATA # #
data = data.drop(data[mask_1 & mask_2].index)
data = data.drop(data[data.currency.isna()].index)

df = data_info(data, 'limpieza_1')
Dicto['limpieza_1'] = df.to_dict(), data.to_dict()

In [5]:
# 4. LIMPIEZA POR COUNTS Y CV
# 4.1 LIMPIEZA POR STATE_NAME EN FUNCION DEL COUNT: NOS QUEDAMOS CON LAS 7 PROVINCIAS/LOCALIDADES CON MAYOR CANTIDAD DE PUBLICACIONES
df = explore_count(data, 'state_name');
df.sort_values(by='total', ascending=False, inplace=True)
lista_state = list(df.index)[:8]
lista = []
for elemento in lista_state:
    lista.append(list(data[data.state_name == elemento].index))
lista_mascara = []
for elemento in lista:
    lista_mascara += elemento

data = data.filter(lista_mascara, axis=0)

# 4.2 LIMPIEZA POR PLACE_NAME EN FUNCION DEL COUNT: NOS QUEDAMOS CON LAS CIUDADES/BARRIOS QUE TENGAN MAS DE 45 PUBLICACIONES \ check:<#df.total.value_counts().to_frame().reset_index().sort_values(by = 'index', ascending=True).head(50)>
lista_filter = []
for ix, state in enumerate(lista_state):
    mascara_iteracion = data.state_name == state
    df = explore_count(data[data.state_name == state], 'place_name');
    mask = df.total > 45
    lista_city = list(df[mask].index)

    lista = []
    for elemento in lista_city:
        lista.append(list(data[mascara_iteracion].loc[data.place_name == elemento].index))

    lista_mascara = []
    for elemento in lista:
        lista_mascara += elemento
    
    lista_filter += lista_mascara
        
data = data.filter(lista_filter, axis=0)

# # GUARDADO DE LA DATA # #
df = data_info(data, 'limpieza_2')
Dicto['limpieza_2'] = df.to_dict(), data.to_dict()

# 4.3 LIMPIEZA POR COEFICIENTE DE VARIACION: NOS QUEDAMOS CON LOS PROPERTY_TYPE(EN FUNCION DE PLACE_NAME) QUE TENGAN MENOS DE 0.99 DE CV EN PRICE_USD_PER_M2
# # CREACION DE DATAFRAME ESTADISTICO # #
df = explore_count(data, 'place_name')
df = explore_count_by(data, 'place_name', 'property_type', df)
df = explore_statics_count(data,'place_name', 'property_type','price_usd_per_m2',df)

#  # FILTROS # #
#nomenclatura: el primer numero hace reperenfencia al tipo de propiedad, y el segundo hace referencia a la cantidad de filtros que lleva implementandose
filtro_1_1 = list(df.columns)[:2]
filtro_2_1 = list(df.columns)[:3:2]
filtro_3_1 = list(df.columns)[:4:3]
filtro_4_1 = list(df.columns)[:5:4]

filtro_1_2 = list(df.columns)[5:8]
filtro_2_2 = list(df.columns)[8:11]
filtro_3_2 = list(df.columns)[11:14]
filtro_4_2 = list(df.columns)[14:17]

filtro_1_3 = list(df.columns)[17:18]
filtro_2_3 = list(df.columns)[18:19]
filtro_3_3 = list(df.columns)[19:20]
filtro_4_3 = list(df.columns)[20:21]

filtro_1 = filtro_1_1 + filtro_1_2 + filtro_1_3
filtro_2 = filtro_2_1 + filtro_2_2 + filtro_2_3
filtro_3 = filtro_3_1 + filtro_3_2 + filtro_3_3
filtro_4 = filtro_4_1 + filtro_4_2 + filtro_4_3

lista_filtros = [filtro_1, filtro_2, filtro_3, filtro_4]

# # CATEGORIZACION # #
# Se crea la columna low_coeficiente_variacion = std/mean
diccionario_prop_type_low_cv = {}

for filtro in lista_filtros: 
    label = str(list(df.loc[:,filtro].columns)[1])
    label = label.replace('count_', '')
    df['LowCV_' + label] = df.loc[:,filtro].eval(str(list(df.loc[:,filtro].columns)[-1])+' < 0.99') #creacion de columna categorica
    mask = df['LowCV_' + label] == True
    diccionario_prop_type_low_cv[label] = list(df.loc[mask,filtro].index)
    
# # FILTRADO # #
# En base a la categoria obtenida se aplica el método de filtrado
set_check = set(list(data.property_type))
lista_dict = list(diccionario_prop_type_low_cv.keys())
lista_filter = []
lista_mascara = []

for ix, propiedad in enumerate(set_check):
    if propiedad == lista_dict[ix]:
        lista_city = diccionario_prop_type_low_cv[propiedad]
        lista = []
        mascara = data.property_type == propiedad
        
        for elemento in lista_city:
            lista.append(list(data.loc[mascara].loc[data.place_name==elemento].index)) # estamos hablando que el tipo de propiedad en la localidad especifica tiene bajos CV, y me quedo con los index

        lista_mascara += lista
for elemento in lista_mascara:
    lista_filter += elemento

data = data.filter(lista_filter, axis=0)

# # GUARDADO DE LA DATA # # 
df = data_info(data, 'limpieza_3')
Dicto['limpieza_3'] = df.to_dict(), data.to_dict()

# 4.4 LIMPIEZA POR COEFICIENTE DE VARIACION: NOS QUEDAMOS CON LOS PROPERTY_TYPE(EN FUNCION DE PLACE_NAME) QUE TENGAN MENOS DE 0.99 DE CV EN SURFACE_TOTAL_IN_M2
# # CREACION DE DATAFRAME ESTADISTICO # #
df = explore_count(data, 'place_name')
df = explore_count_by(data, 'place_name', 'property_type', df)
df = explore_statics_count(data,'place_name', 'property_type','surface_total_in_m2',df)

#  # FILTROS # #
#nomenclatura: el primer numero hace reperenfencia al tipo de propiedad, y el segundo hace referencia a la cantidad de filtros que lleva implementandose
filtro_1_1 = list(df.columns)[:2]
filtro_2_1 = list(df.columns)[:3:2]
filtro_3_1 = list(df.columns)[:4:3]
filtro_4_1 = list(df.columns)[:5:4]


filtro_1_2 = list(df.columns)[5:8]
filtro_2_2 = list(df.columns)[8:11]
filtro_3_2 = list(df.columns)[11:14]
filtro_4_2 = list(df.columns)[14:17]


filtro_1_3 = list(df.columns)[17:18]
filtro_2_3 = list(df.columns)[18:19]
filtro_3_3 = list(df.columns)[19:20]
filtro_4_3 = list(df.columns)[20:21]


filtro_1 = filtro_1_1 + filtro_1_2 + filtro_1_3
filtro_2 = filtro_2_1 + filtro_2_2 + filtro_2_3
filtro_3 = filtro_3_1 + filtro_3_2 + filtro_3_3
filtro_4 = filtro_4_1 + filtro_4_2 + filtro_4_3

lista_filtros = [filtro_1, filtro_2, filtro_3, filtro_4]

# # CATEGORIZACION # #
# Se crea la columna low_coeficiente_variacion = std/mean
diccionario_prop_type_low_cv = {}

for filtro in lista_filtros: 
    label = str(list(df.loc[:,filtro].columns)[1])
    label = label.replace('count_', '')
    df['LowCV_' + label] = df.loc[:,filtro].eval(str(list(df.loc[:,filtro].columns)[-1])+' < 0.99') #creacion de columna categorica
    mask = df['LowCV_' + label] == True
    #Dicto
    diccionario_prop_type_low_cv[label] = list(df.loc[mask,filtro].index)
    
# # FILTRADO # #
# En base a la categoria obtenida se aplica el método de filtrado
set_check = set(list(data.property_type))
lista_dict = list(diccionario_prop_type_low_cv.keys())
lista_filter = []
lista_mascara = []

for ix, propiedad in enumerate(set_check):
    if propiedad == lista_dict[ix]:
        lista_city = diccionario_prop_type_low_cv[propiedad]
        lista = []
        mascara = data.property_type == propiedad
        
        for elemento in lista_city:
            lista.append(list(data.loc[mascara].loc[data.place_name==elemento].index)) 

        lista_mascara += lista
for elemento in lista_mascara:
    lista_filter += elemento

data = data.filter(lista_filter, axis=0)

# # GUARDADO DE LA DATA # # 
df = data_info(data, 'limpieza_4')
Dicto['limpieza_4'] = df.to_dict(), data.to_dict()

In [6]:
# 5. ELIMINACION DE OUTLIERS POR SURFACE_COVERED_IN_M2
print ('EXCLUSION DE OUTLIERS EN SUPERFICIE')

data_aux = data.copy()
lista = [data_aux, data]

lista_rangos_main = []
lista_drops_main = []

for ix, data_sup in enumerate(lista):
    
    serie_check = data_sup.property_type
    set_check = set(list(serie_check))
    lista_rangos = []
    lista_drops = []
    if ix == 0: print ('='*20,'\nvalores data_aux\n', '='*20)
    if ix == 1: print ('='*20,'\nvalores data\n', '='*20)
        
    for i,elemento in enumerate (set_check):
        print ('-----------------Iteracion en {}-----------------'.format(elemento))
        mascara_iter_1 = serie_check == elemento
        
        if ix == 1: #data
            if elemento == 'house':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'surface_total_in_m2', [0.5, 97.5])
            if elemento == 'apartment':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'surface_total_in_m2', [0.5, 97.5])
            if elemento == 'store':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'surface_total_in_m2', [0.5, 97.5])
            if elemento == 'PH':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'surface_total_in_m2', [0.5, 97.5])
                
        if ix==0:# data_aux
            if elemento == 'PH':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'surface_covered_in_m2', [1, 99])
            else:
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'surface_covered_in_m2', [2.5, 97.5])
    
        lista_rangos.append(rango_outlier)
        # IMPUTACION COLUMNA        
        if ix == 0:
            serie_outlier = data_sup.surface_total_in_m2.loc[mascara_iter_1]
            mask_data = data_sup.loc[serie_outlier.index, 'surface_total_in_m2'].dropna()
            indice_outlier = mascara_outlier_index_list(data_sup.loc[mask_data.index], 'surface_total_in_m2', rango_outlier)    
        if ix == 1:
            serie_outlier = data_sup.surface_total_in_m2.loc[mascara_iter_1]
            mask_data = data_sup.loc[serie_outlier.index, 'surface_total_in_m2'].dropna()
            indice_outlier = mascara_outlier_index_list(data_sup.loc[mask_data.index], 'surface_total_in_m2', rango_outlier)    
        
        lista_drops.append(indice_outlier)

    lista_rangos_main.append(lista_rangos)   
    lista_drops_main.append(lista_drops)
    for lista_indices in lista_drops:
        data_sup.drop(lista_indices, inplace=True)

    print ('--------------------------------------------------------------------------------\n')   
    lista[ix] = data_sup

# # GUARDADO DE LA DATA # #    
df = data_info(lista[0], 'limpieza_5_aux')
Dicto['limpieza_5_aux'] = df.to_dict(), lista[0].to_dict()

df = data_info(lista[1], 'without_outliers_surface')
data = lista[1]
Dicto['limpieza_5'] = df.to_dict(), lista[1].to_dict()    
    
# 6. RECUPERACION DE DATOS SURFACE_COVERED:

print ('\nRECUPERO DATA: surface_covered_in_m2\n')

for ix, data_sup in enumerate(lista):
    data_sup['surface_recovery'] = data_sup['price'] / data_sup['price_usd_per_m2']

    serie_check = data_sup['surface_recovery']
    serie_check.dropna(inplace=True)
    mask_data = data_sup.surface_total_in_m2.isna()
            
    for i, propiedad in enumerate(set_check):
        data_check = data_sup.loc[mask_data].loc[mascara_iter_1]
        lista_drop = check_range(data_check, 'surface_recovery', lista_rangos_main[ix][i])
    
        data_sup.drop(lista_drop, inplace=True)
   
    # RECUPERO DE LA DATA    
    data_sup.surface_covered_in_m2.fillna(data_sup.surface_recovery, inplace=True)
    data_sup.drop('surface_recovery', axis=1, inplace=True)

    lista[ix] = data_sup

# # GUARDADO DEL A DATA # #    
df = data_info(lista[0],'recupero_5')    
Dicto['recupero_5'] = df.to_dict(), lista[0].to_dict()

data = lista[1]

EXCLUSION DE OUTLIERS EN SUPERFICIE
valores data_aux
-----------------Iteracion en house-----------------
range_outlier: type: <class 'list'>, len: 2, value: [60.0, 560.0]
-----------------Iteracion en store-----------------
range_outlier: type: <class 'list'>, len: 2, value: [16.95, 1515.4499999999946]
-----------------Iteracion en PH-----------------
range_outlier: type: <class 'list'>, len: 2, value: [20.0, 260.0]
-----------------Iteracion en apartment-----------------
range_outlier: type: <class 'list'>, len: 2, value: [26.0, 220.0]
--------------------------------------------------------------------------------

valores data
-----------------Iteracion en house-----------------
range_outlier: type: <class 'list'>, len: 2, value: [54.205, 1464.5749999999985]
-----------------Iteracion en store-----------------
range_outlier: type: <class 'list'>, len: 2, value: [16.86, 1169.9999999999989]
-----------------Iteracion en PH-----------------
range_outlier: type: <class 'list'>, len: 2,

In [7]:
# 7 ELIMINACION DE OUTLIERS DE PRICE
print ('EXCLUSION DE OUTLIERS PRICE\n') #APLICAR OUTLIERS EN PRICE PERO NO EN EL TIPO DE CATEGORIA PH, CORTAMOS SOLO LA COLA DERECHA AL 1%, APLICAR A LOS DOS DATASETS

data_aux = pd.DataFrame(Dicto['recupero_5'][1])
data = pd.DataFrame(Dicto['limpieza_5'][1])
lista = [data_aux, data]

for ix, data_sup in enumerate(lista):
    
    serie_check = data_sup.property_type
    set_check = set(list(serie_check))
    lista_rangos = []
    lista_drops = []
    if ix == 0: print ('='*20,'\nvalores data_aux\n', '='*20)
    if ix == 1: print ('='*20,'\nvalores data\n', '='*20)
    for i,elemento in enumerate (set_check):
        print ('-----------------Iteracion en {}-----------------'.format(elemento))
        
        mascara_iter_1 = serie_check == elemento
        
        if ix == 1: #data
            if elemento == 'PH':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'price_usd_per_m2', [0.01, 99.9])
            if elemento == 'apartment' or elemento == 'store':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'price_usd_per_m2', [0.01, 95])
            if elemento == 'house':
                rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'price_usd_per_m2', [0.01, 97.5])
       
        if ix == 0: # data_price
            rango_outlier = rango_extremos(data_sup.loc[mascara_iter_1], 'price_usd_per_m2', [0.01, 95])
        
        lista_rangos.append(rango_outlier)
        serie_outlier = data_sup['price_usd_per_m2'].loc[mascara_iter_1]
        mask_data = data_sup.loc[serie_outlier.index, 'price_usd_per_m2'].dropna()
        indice_outlier = mascara_outlier_index_list(data_sup.loc[mask_data.index], 'price_usd_per_m2', rango_outlier)    
        lista_drops.append(indice_outlier)
        
    for lista_indices in lista_drops:
        data_sup.drop(lista_indices, inplace=True)

    print ('--------------------------------------------------------------------------------\n')    
    lista[ix] = data_sup
    
# # GUARDADO DE LA DATA # #
df = data_info(lista[0], 'limpieza_6_aux')
Dicto['limpieza_6_aux'] = df.to_dict(), lista[0].to_dict()

df = data_info(lista[1], 'without_outliers_price')
data = lista[1]
Dicto['limpieza_6'] = df.to_dict(), lista[1].to_dict() 


##################################################
#falta
# 1. revisar el extremo izquierdo de los datos de precio y superficie
# 2. tirar los low count de property type en funcion de state_name
# 3. Analizar una forma de normalizar los datos en funcion de property_type y state_name (o armar subconjuntos)
# 4. graficar


EXCLUSION DE OUTLIERS PRICE

valores data_aux
-----------------Iteracion en house-----------------
range_outlier: type: <class 'list'>, len: 2, value: [31.33454039434251, 2703.8747931605076]
-----------------Iteracion en store-----------------
range_outlier: type: <class 'list'>, len: 2, value: [30.28517786666666, 5477.901266477125]
-----------------Iteracion en PH-----------------
range_outlier: type: <class 'list'>, len: 2, value: [60.32597278015041, 2805.5861526357203]
-----------------Iteracion en apartment-----------------
range_outlier: type: <class 'list'>, len: 2, value: [67.65300772486772, 4266.835670731707]
--------------------------------------------------------------------------------

valores data
-----------------Iteracion en house-----------------
range_outlier: type: <class 'list'>, len: 2, value: [32.86051427813456, 3249.627483443707]
-----------------Iteracion en store-----------------
range_outlier: type: <class 'list'>, len: 2, value: [30.245118142857137, 5479.45205

In [10]:
data.property_type.value_counts()

apartment    17904
house         3239
PH            2619
store          209
Name: property_type, dtype: int64

In [9]:
error
# 8 ELIMINAR LAS ENTRADAS EN DONDE SURFACE_COVERED_PRICE_INDEX SEA VALOR LEJANO A 1 Y QUE NO SEA NAN SUPERFICIE_TOTAL
### VOLVER A APLICAR CRITERIO DE DROP PROPERTY TYPE EN FUNCION DE COUNT POR STATE_NAME (!) (!) (!)
#for ix, data_sup in enumerate(lista):
#    data_sup['pct_surf_cov_vs_tot'] = data_sup.surface_covered_in_m2 / data_sup.surface_total_in_m2
#    data_sup.loc[data_sup.pct_surf_cov_vs_tot == 1, 'surface_total_in_m2'] = np.nan
#    data_sup.drop('pct_surf_cov_vs_tot', axis=1, inplace=True)
#    lista[ix] = data_sup


plt.plot(data.price_usd_per_m2, data.surface_total_in_m2, '+');
plt.legend(numpoints=1);
ax = plt.axes()
ax.set(xlabel='price_usd_per_m2', ylabel='surface_total_in_m2', title='(x,y) = (price_usd_per_m2, surface_total_in_m2)')

NameError: name 'error' is not defined

In [ ]:
lista_marker = ['o','.',',','x','+','v','^', '<', '>', 's', 'd']

#fig_kw = dict(autoscalex_on= True)
fig, ax = plt.subplots(2, figsize = (10,4))#,**fig_kw)

for i, elemento in enumerate(list(data.state_name.unique())):
    x = data[data.state_name==elemento].surface_total_in_m2
    y = data[data.state_name==elemento].state_name
    #ax[1].plot(x, y, lista_marker[i], label='{}= {}'.format(elemento, lista_marker[i]))
    #plt.legend(numpoints=1)
    ax[0] = plt.axes()
    ax[0].set(xlabel='surface_total_in_m2', ylabel='state_name')
    ax[0].plot(x, y, lista_marker[i], label='{}'.format(elemento))
    ax[0].axis='equal'
    #####ax[0].set_xticks(ticks=[], minor=True)
    leg = ax[0].legend(loc='upper left', fancybox=True, framealpha=1, shadow=True, borderpad=1)#frameon=False)
    #ax[0].legend(numpoints=1)
plt.show()

#fig, ax = plt.subplots(2, figsize = (10,4))
for i, elemento in enumerate(list(data.state_name.unique())):
    x = data[data.state_name==elemento].price_usd_per_m2
    y = data[data.state_name==elemento].state_name
    ax[1] = plt.axes()
    ax[1].set(xlabel='price_usd_per_m2', ylabel='state_name')
    ax[1]
    ax[1].plot(data[data.state_name==elemento].price_usd_per_m2, data[data.state_name==elemento].state_name, lista_marker[i], label='{}= {}'.format(elemento, lista_marker[i]))
    #plt.legend(numpoints=1)

    
plt.show()

In [ ]:
x1 = data[data.property_type == 'apartment']
x2 = data[data.property_type == 'house']
x3 = data[data.property_type == 'store']
x4 = data[data.property_type == 'PH']

sns.set()
fig = plt.figure(figsize=(16,6))

kwargs= dict(title='histograma surface_total_in_m2 x tipo de propiedad', xlabel='surface_total_in_m2', ylabel='count')#xlim=(0,2500)
ax[0] = plt.axes(**kwargs)

kwargs = dict(histtype='stepfilled', alpha=0.3, bins='auto', align='mid')
ax[0].hist(x1.surface_total_in_m2, color='green', label='apartment', density=True, **kwargs)
ax[0].hist(x2.surface_total_in_m2, color='gold', label='house', density=True, **kwargs)
ax[0]=plt.hist(x3.surface_total_in_m2, color='blue', label='store', density=True, **kwargs);
ax[0]= plt.hist(x4.surface_total_in_m2, color='red', label='PH', density=True, **kwargs);
plt.legend();
plt.show()

fig = plt.figure(figsize=(16,6))

kwargs= dict(title='histograma price_usd_per_m2 x tipo de propiedad', xlabel='price_usd_per_m2', ylabel='count')# yscale='log', , xlim=(0,10000)
ax[1] = plt.axes(**kwargs)

kwargs = dict(histtype='stepfilled', alpha=0.3, bins='auto', align='mid')
ax[1] =plt.hist(x1.price_usd_per_m2, color='green', label='apartment', density=True, **kwargs)
ax[1] =plt.hist(x2.price_usd_per_m2, color='gold', label='house', density=True, **kwargs)
ax[1] =plt.hist(x3.price_usd_per_m2, color='blue', label='store', density=True, **kwargs);
ax[1] =plt.hist(x4.price_usd_per_m2, color='red', label='PH', density=True, **kwargs);
plt.legend();

plt.show()

In [ ]:
x1 = data[data.property_type == 'apartment']
x2 = data[data.property_type == 'house']
x3 = data[data.property_type == 'store']
x4 = data[data.property_type == 'PH']

sns.set(style = 'whitegrid')
fig = plt.figure(figsize=(16,6))

kwargs= dict(title='histograma surface_total_in_m2 x tipo de propiedad', xlabel='surface_total_in_m2', yscale='log', ylabel='count')#xlim=(0,2500), 
ax[0] = plt.axes(**kwargs)

kwargs = dict(histtype='stepfilled', alpha=0.3, bins='auto', align='mid')
ax[0].hist(x1.surface_total_in_m2, color='green', label='apartment', **kwargs)#, density=True
ax[0].hist(x2.surface_total_in_m2, color='gold', label='house', **kwargs)
ax[0]=plt.hist(x3.surface_total_in_m2, color='blue', label='store', **kwargs);
ax[0]= plt.hist(x4.surface_total_in_m2, color='red', label='PH', **kwargs);
plt.legend();
plt.show()

fig = plt.figure(figsize=(16,6))

kwargs= dict(title='histograma price_usd_per_m2 x tipo de propiedad', xlabel='price_usd_per_m2', yscale='log', ylabel='count') #, xlim=(0,20000)
ax[1] = plt.axes(**kwargs)

kwargs = dict(histtype='stepfilled', alpha=0.3, bins='auto', align='mid')
ax[1] =plt.hist(x1.price_usd_per_m2, color='green', label='apartment', density=True, **kwargs)
ax[1] =plt.hist(x2.price_usd_per_m2, color='gold', label='house', density=True, **kwargs)
ax[1] =plt.hist(x3.price_usd_per_m2, color='blue', label='store', density=True, **kwargs);
ax[1] =plt.hist(x4.price_usd_per_m2, color='red', label='PH', density=True, **kwargs);
plt.legend();

plt.show()

In [ ]:
col = [data.property_type == 'apartment', data.property_type == 'house', data.property_type == 'store', data.property_type == 'PH']
sns.set()

for mascara in col:
    sns.kdeplot(data[mascara].price_usd_per_m2, shade=True)

In [ ]:
#help(plt.axes);
#help(plt.figure);
#help(axes.set_xticks);
#help(sns.FacetGrid.map_dataframe);
#help(sns.FacetGrid.map);
#help(plt.hist)
#help(sns.kdeplot)
#help(sns.pairplot)
#help(sns.set)

In [ ]:

# 9 MODULO DE VISUALIZACION NANS, BOXPLOT Y SCATTERPLOT
print ('\n', '='*40,'pct_nan: porcentaje nans en cada columna','='*40)
df = pd.DataFrame(Dicto['data_cruda'][0])
fig, ax = plt.subplots(figsize=(22, 6))
ax.grid(axis='x')
axes= plt.gca()
axes.set_xlim(0,100)
plt.barh(df.columna, df.pct_nan, label='porcentaje de NaN\'s')

df = pd.DataFrame(Dicto['limpieza_1'][0])
fig, ax = plt.subplots(figsize=(22, 6))
ax.grid(axis='x')
axes= plt.gca()
axes.set_xlim(0,100)
plt.barh(df.columna, df.pct_nan, label='porcentaje de NaN\'s')

df = pd.DataFrame(Dicto['data5'][0])
fig, ax = plt.subplots(figsize=(22, 6))
ax.grid(axis='x')
axes= plt.gca()
axes.set_xlim(0,100)
plt.barh(df.columna, df.pct_nan, label='porcentaje de NaN\'s')

df = pd.DataFrame(Dicto['data7'][0])
print ('\n', '='*40,'pct_nan: porcentaje nans en cada columna','='*40)
fig, ax = plt.subplots(figsize=(22, 6))
ax.grid(axis='x')
axes= plt.gca()
axes.set_xlim(0,100)
plt.barh(df.columna, df.pct_nan, label='porcentaje de NaN\'s')

In [ ]:
# 14 CREAR CATEGORIA OPR ENCIMA Y POR DEBAJO DE LA MEDIANA

In [ ]:
# 14 MODULO DE CREACION DE SUBCONJUNTO data info y explore del criterio
4 datasets uno por tipo de propiedad
1 dataset armado con un primer criterio